In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("C:/Users/N/Desktop/Online retail business Project/OnlineRetail.csv", encoding="ISO-8859-1", dtype={'CustomerID': str})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


In [4]:
df['totalamt'] = df['Quantity'] * df['UnitPrice']
df['iscancllation'] = 0
df.loc[df['InvoiceNo'].str.startswith('C'), 'iscancllation'] = 1

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,totalamt,iscancllation
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,15.30,0
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,22.00,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34,0


In [8]:
df[df['CustomerID'].isnull()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,totalamt,iscancllation
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom,0.00,0
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom,2.51,0
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom,5.02,0
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom,3.40,0
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom,3.32,0


In [12]:
# Remove quatity < 0 and Unit price < 0
df[df['Quantity'] < 0].sample(20)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,totalamt,iscancllation
210958,C555329,23110,PARISIENNE KEY CABINET,-1,6/2/2011 11:08,5.75,15622,United Kingdom,-5.75,1
497443,C578371,22414,DOORMAT NEIGHBOURHOOD WITCH,-1,11/24/2011 11:16,8.25,14410,United Kingdom,-8.25,1
418904,C572754,72802C,VANILLA SCENT CANDLE JEWELLED BOX,-36,10/25/2011 17:29,1.00,15482,United Kingdom,-36.00,1
81799,C543175,22168,ORGANISER WOOD ANTIQUE WHITE,-2,2/4/2011 9:46,8.50,12712,Germany,-17.00,1
382775,C569945,22720,SET OF 3 CAKE TINS PANTRY DESIGN,-2,10/6/2011 18:14,4.95,14141,United Kingdom,-9.90,1
536922,C581229,23158,SET OF 5 LUCKY CAT MAGNETS,-36,12/8/2011 10:14,2.08,12558,USA,-74.88,1
32751,C539114,22667,RECIPE BOX RETROSPOT,-3,12/16/2010 11:24,2.95,12494,France,-8.85,1
201896,C554344,22064,PINK DOUGHNUT TRINKET POT,-1,5/23/2011 17:44,1.65,14032,United Kingdom,-1.65,1
414804,C572488,22972,CHILDREN'S SPACEBOY MUG,-1,10/24/2011 13:57,1.65,12474,Germany,-1.65,1
465284,C576216,85123A,WHITE HANGING HEART T-LIGHT HOLDER,-3,11/14/2011 12:56,2.95,12584,Italy,-8.85,1
